https://blog.csdn.net/qq_24951479/article/details/132495408

第十一周作业：
1. 参考课堂案例，使用指定的数据集，编写代码实现ner模型训练和推流。
https://huggingface.co/datasets/doushabao4766/msra_ner_k_V3
2. 完成预测结果的实体抽取。
    输入：“双方确定了今后发展中美关系的指导方针。”
    输出：[{"entity":"ORG","content":"中"},{"entity":"ORG","content":"美"}]
3. 整理Dataset、Trainer、TrainingArgument、DataCollator、Evaluate 知识点，总结文档（无需提交）

In [3]:
from transformers import AutoModelForTokenClassification , AutoTokenizer
from transformers import TrainingArguments, Trainer
import torch
import evaluate  # pip install evaluate
import seqeval  # pip install seqeval
import torch
from datasets import load_dataset

2025-06-04 23:44:51.780137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749080692.263168      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749080692.375708      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 构建分词器

In [4]:
# 构建分词器
tokenizer = AutoTokenizer.from_pretrained(
    'google-bert/bert-base-chinese' , 
    max_length = 91 , 
    padding='max_length',  # 填充到最大长度
    truncation=True,    # 超过长度则截断
    return_tensors='pt')
tokenizer

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

BertTokenizerFast(name_or_path='google-bert/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [5]:
tokenizer

BertTokenizerFast(name_or_path='google-bert/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [6]:
# 加载数据集
ds = load_dataset('doushabao4766/msra_ner_k_V3')

README.md:   0%|          | 0.00/697 [00:00<?, ?B/s]

(…)-00000-of-00001-42717a92413393f9.parquet:   0%|          | 0.00/13.9M [00:00<?, ?B/s]

(…)-00000-of-00001-8899cab5fdab45bc.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3443 [00:00<?, ? examples/s]

In [7]:
for row in ds["test"]:
    print(row['tokens'])
    print(row['ner_tags'])
    print(row['knowledge'])
    break

['中', '共', '中', '央', '致', '中', '国', '致', '公', '党', '十', '一', '大', '的', '贺', '词', '各', '位', '代', '表', '、', '各', '位', '同', '志', '：', '在', '中', '国', '致', '公', '党', '第', '十', '一', '次', '全', '国', '代', '表', '大', '会', '隆', '重', '召', '开', '之', '际', '，', '中', '国', '共', '产', '党', '中', '央', '委', '员', '会', '谨', '向', '大', '会', '表', '示', '热', '烈', '的', '祝', '贺', '，', '向', '致', '公', '党', '的', '同', '志', '们', '致', '以', '亲', '切', '的', '问', '候', '！']
[3, 4, 4, 4, 0, 3, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
同志：同志可以指：志同道合、有志一同的人同志(政治)：清末民初以来，革命党人、共产党人互称彼此的称谓。在社会主义国家也用于人与人之间的称呼。同志(LGBT)：同性恋者的代称。有时也泛称同性恋、双性恋、跨性别等LGBT群体。


# 获取实体映射词典

In [8]:
label_list = ds["train"].features["ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

# 数据预处理函数

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],
                                 is_split_into_words=True , 
                                 max_length = 91 , 
                                 padding='max_length',  # 填充到最大长度
                                 truncation=True,    # 超过长度则截断
                                 return_tensors='pt')
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = ds.map(tokenize_and_align_labels, batched=True)


tokenized_datasets
    

Map:   0%|          | 0/45001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 45001
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'knowledge', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3443
    })
})

# 加载模型

In [10]:
# 获取模型
model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese' , num_labels = len(label_list))

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 使用TrainingArguments 构建训练参数

In [11]:
args = TrainingArguments(
    output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
    num_train_epochs = 3,    # 训练 epoch
    save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
    per_device_train_batch_size=64,  # 训练批次
    per_device_eval_batch_size=64,
    report_to='tensorboard',  # 训练输出记录
    eval_strategy="epoch",
)

# 构建 Train

In [12]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)


# 模型训练以及保存模型

In [13]:
trainer.train()

# 保存模型
model.save_pretrained("./ner_model")
tokenizer.save_pretrained("./ner_model")

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.023258
2,0.041100,0.023055
3,0.009000,0.023872


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


('./ner_model/tokenizer_config.json',
 './ner_model/special_tokens_map.json',
 './ner_model/vocab.txt',
 './ner_model/added_tokens.json',
 './ner_model/tokenizer.json')

------------
# 实体抽取

In [19]:
# 实体抽取函数
def exttract_entities(text):
    # 加载模型与分词器
    model = AutoModelForTokenClassification.from_pretrained('./ner_model')
    tokenizer = AutoTokenizer.from_pretrained('./ner_model')

    # 分词和预测
    inputs = tokenizer(text , return_tensors = 'pt' , truncation = True)
    with torch.no_grad():
        outputs = model(**inputs)

    # 获取预测结果
    predictions = torch.argmax(outputs.logits , dim = 2)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    predicted_labels = [label_list[p] for p in predictions[0].numpy()]

    # 提取实体
    entities = []
    current_entity = None
    for token, label in zip(tokens, predicted_labels):
        if label.startswith("B-"):
            if current_entity:
                entities.append(current_entity)
            current_entity = {"entity": label[2:], "content": token.replace("##", "")}
        elif label.startswith("I-") and current_entity:
            current_entity["content"] += token.replace("##", "")
        else:
            if current_entity:
                entities.append(current_entity)
                current_entity = None
    
    return entities

In [20]:
# 测试示例
text = "双方确定了今后发展中美关系的指导方针。"
print(exttract_entities(text))

[{'entity': 'LOC', 'content': '中'}, {'entity': 'LOC', 'content': '美'}]
